## Common Use Scinarios for Deep Learning Models

### 1. Building Models on Training Data
1. Customize Network Layout: e.g., number of layers, connections, activation functions, weight initializations
2. Customize Cost Function: special regularization
3. Choose or Customize optimization method, e.g., l-bfgs, cg, batch-gd, sgd, etc.
4. Specify early stopping, train/validation data, performance monitoring
5. Save learned weights and be able to recover from it

### 2. Use Models on New Data
1. Restore model from its trained weights and yaml configuration
2. Predict on new data
3. Extract weights from hidden layers as features - mostly only useful for models pre-trained on large data, e.g., Caffe, Overfeat, sklearn-theano

[Machine Learning Using Pylearn2](https://blog.safaribooksonline.com/2014/02/10/pylearn2-regression-3rd-party-data/)


**we will try to cover these points in this notebook**
**use the [data-downloading script](https://github.com/lisa-lab/DeepLearningTutorials/blob/master/data/download.sh) to download the necessary data into `./data` folder** 

## Pylearn2 for Deep Learning
1. pylearn2 to Theano is similiar as scipy to numpy
2. it utilizes yaml for quick experiemnt setup, under a unified framework including `dataset`, `algorithm`(optimizer), `model` (network), which are leigo blocks for deep leanring
3. in the yaml configuration, you can use `!obj:` to create instance (composite of both data and methods), `!import` to attach to customized functions (e.g., cost function), and `!pkl:` to load data 

In [1]:
!pip show pylearn2

---
Metadata-Version: 1.0
Name: pylearn2
Version: 0.1.dev0
Summary: A machine learning library built on top of Theano.
Home-page: UNKNOWN
Author: UNKNOWN
Author-email: UNKNOWN
License: BSD 3-clause license
Location: /home/dola/opt/pylearn2
Requires: numpy, pyyaml, argparse, Theano


In [1]:
from pylearn2.config import yaml_parse
from pylearn2.datasets import DenseDesignMatrix
import cPickle
from sklearn import preprocessing

Couldn't import dot_parser, loading of dot files will not be possible.


### 1. [Basic Example of Building Softmax Regression for MNIST - with YAML](http://nbviewer.ipython.org/github/lisa-lab/pylearn2/blob/master/pylearn2/scripts/tutorials/softmax_regression/softmax_regression.ipynb)

**The exercise focuses on how easy or hard to quickly test a model on an numpy.array in pylearn2**

**its use feels quite counter-intuitive for explorative data science - it needs to force everything into YAML and dump everything on disk first - Theano has a much more friendly interface when integrated with other python objects in this case**

**but pylearn2 may have its point of doing this as it uses configuration to train pre-defined data to get good performance, but not necessarily in an explorative environment**


In [3]:
## prepare data - from single mnist.pkl to create training, valiadtion and testing set and pickle them
## even though pylearn2.datasets.DenseDesignMatrix accepts integer array as y, but SoftmaxRegression Model
## only accepts one-hot encoding
mnist_data_config = r"!pkl: '../data/mnist.pkl'"
train_mnist, valid_mnist, test_mnist = yaml_parse.load(mnist_data_config)
coder = preprocessing.OneHotEncoder()
train_mnist = DenseDesignMatrix(X = train_mnist[0], y = coder.fit_transform(train_mnist[1].reshape((-1, 1))).toarray())
valid_mnist = DenseDesignMatrix(X = valid_mnist[0], y = coder.fit_transform(valid_mnist[1].reshape((-1, 1))).toarray())
test_mnist = DenseDesignMatrix(X = test_mnist[0], y = coder.fit_transform(test_mnist[1].reshape((-1, 1))).toarray())
cPickle.dump(train_mnist, open("../data/train_mnist.pkl", "w"))
cPickle.dump(valid_mnist, open("../data/valid_mnist.pkl", "w"))
cPickle.dump(test_mnist, open("../data/test_mnist.pkl", "w"))

In [4]:
!ls ../data

images	   mnist.pkl.gz       test_mnist.pkl   valid_mnist.pkl
mnist.pkl  mnist_py3k.pkl.gz  train_mnist.pkl


In [5]:
dataset_config = r"""&train !pkl: '../data/train_mnist.pkl'
"""

model_config = r"""!obj:pylearn2.models.softmax_regression.SoftmaxRegression {
  n_classes: 10,
  irange: 0.,
  nvis: 784
}"""


algorithm_config = r"""!obj:pylearn2.training_algorithms.bgd.BGD {
  batch_size: 10000,
  conjugate: 1,
  monitoring_dataset: {
    'train': *train,
    'valid': !pkl: '../data/valid_mnist.pkl'
  },
  termination_criterion: !obj:pylearn2.termination_criteria.MonitorBased {
    channel_name: "valid_y_misclass"
  }
}
"""

driver_config = r"""!obj:pylearn2.train.Train {
  dataset: %s
  , model: %s
  , algorithm: %s
  , extensions: [!obj:pylearn2.train_extensions.best_params.MonitorBasedSaveBest {
             channel_name: 'valid_y_misclass',
             save_path: "../models/softmax_regression_best.pkl"
        },]
  , save_path: "../models/softmax_regression.pkl"
  , save_freq: 1
}
""" % (dataset_config, model_config, algorithm_config)



In [6]:
%%capture log
driver = yaml_parse.load(driver_config)
driver.main_loop()

In [7]:
print log.stdout[-500:]

9996146
	valid_y_mean_max_class: 0.917966512741
	valid_y_min_max_class: 0.248488289737
	valid_y_misclass: 0.0714
	valid_y_nll: 0.261678336848
	valid_y_row_norms_max: 1.99803194598
	valid_y_row_norms_mean: 0.574710145358
	valid_y_row_norms_min: 0.0
Saving to ../models/softmax_regression.pkl...
Saving to ../models/softmax_regression.pkl done. Time elapsed: 0.010190 seconds
Saving to ../models/softmax_regression.pkl...
Saving to ../models/softmax_regression.pkl done. Time elapsed: 0.009725 seconds



In [8]:
## make use of trained model
## You need to do all the geeky stuff here to convert it back to Theano
import theano.tensor as T
import theano
import numpy as np 
softmax_model = cPickle.load(open("../models/softmax_regression.pkl"))
X = softmax_model.get_input_space().make_theano_batch()
y = softmax_model.fprop(X)
ylabel = T.argmax(y, axis = 1)
predict = theano.function([X], ylabel)

yhat = predict(test_mnist.X)
ytarget = cPickle.load(open("../data/mnist.pkl"))[-1][1]
print "classification rate on test data", np.mean(yhat == ytarget)

classification rate on test data 0.924


### 2. [stacked autoencoders example by yaml](http://nbviewer.ipython.org/github/lisa-lab/pylearn2/blob/master/pylearn2/scripts/tutorials/stacked_autoencoders/stacked_autoencoders.ipynb)

1. layerwise pre-training (unsupervised) for denoising autoencoder
2. stacking these layers to form a MLP and fine tune it with supverised learning

In [9]:
%%capture log

## layer 1 - unsupervised training
layer1_yaml = r"""
!obj:pylearn2.train.Train {
    dataset: &train !pkl: '../data/train_mnist.pkl',
    model: !obj:pylearn2.models.autoencoder.DenoisingAutoencoder {
        nvis : 784,
        nhid : 500,
        irange : 0.05,
        corruptor: !obj:pylearn2.corruption.BinomialCorruptor {
            corruption_level: .2,
        },
        act_enc: "tanh",
        act_dec: null,    # Linear activation on the decoder side.
    },
    algorithm: !obj:pylearn2.training_algorithms.sgd.SGD {
        learning_rate : 1e-3,
        batch_size : 100,
        monitoring_batches : 5,
        monitoring_dataset : *train,
        cost : !obj:pylearn2.costs.autoencoder.MeanSquaredReconstructionError {},
        termination_criterion : !obj:pylearn2.termination_criteria.EpochCounter {
            max_epochs: 10,
        },
    },
    save_path: "../models/dae_l1.pkl",
    save_freq: 1
}
"""
layer1 = yaml_parse.load(layer1_yaml)


%time layer1.main_loop()

In [10]:
print log.stdout[-500:]

 ../models/dae_l1.pkl done. Time elapsed: 0.577441 seconds
Time this epoch: 12.299725 seconds
Monitoring step:
	Epochs seen: 10
	Batches seen: 5000
	Examples seen: 500000
	learning_rate: 0.001
	objective: 11.8870911208
	total_seconds_last_epoch: 16.161059
	training_seconds_this_epoch: 12.299725
Saving to ../models/dae_l1.pkl...
Saving to ../models/dae_l1.pkl done. Time elapsed: 0.574657 seconds
Saving to ../models/dae_l1.pkl...
Saving to ../models/dae_l1.pkl done. Time elapsed: 0.538063 seconds



In [11]:
%%capture log
## second layer trainning - 2nd layer takes the output of the 1st layer as its input
layer2_yaml = r"""!obj:pylearn2.train.Train {
    dataset: &train !obj:pylearn2.datasets.transformer_dataset.TransformerDataset {
        raw: !pkl: '../data/train_mnist.pkl',
        transformer: !pkl: "../models/dae_l1.pkl" # use layer 1 as input
    },
    model: !obj:pylearn2.models.autoencoder.DenoisingAutoencoder {
        nvis : 500,
        nhid : 500,
        irange : 0.05,
        corruptor: !obj:pylearn2.corruption.BinomialCorruptor {
            corruption_level: .3,
        },
        act_enc: "tanh",
        act_dec: null,    # Linear activation on the decoder side.
    },
    algorithm: !obj:pylearn2.training_algorithms.sgd.SGD {
        learning_rate : 1e-3,
        batch_size : 100,
        monitoring_batches : 5,
        monitoring_dataset : *train,
        cost : !obj:pylearn2.costs.autoencoder.MeanSquaredReconstructionError {},
        termination_criterion : !obj:pylearn2.termination_criteria.EpochCounter {
            max_epochs: 10,
        },
    },
    save_path: "../models/dae_l2.pkl",
    save_freq: 1
}
"""

layer2 = yaml_parse.load(layer2_yaml)


%time layer2.main_loop()

In [12]:
print log.stdout[-500:]

o ../models/dae_l2.pkl done. Time elapsed: 0.407241 seconds
Time this epoch: 12.709962 seconds
Monitoring step:
	Epochs seen: 10
	Batches seen: 5000
	Examples seen: 500000
	learning_rate: 0.001
	objective: 4.3132512472
	total_seconds_last_epoch: 16.045796
	training_seconds_this_epoch: 12.709962
Saving to ../models/dae_l2.pkl...
Saving to ../models/dae_l2.pkl done. Time elapsed: 0.424044 seconds
Saving to ../models/dae_l2.pkl...
Saving to ../models/dae_l2.pkl done. Time elapsed: 0.397842 seconds



In [13]:
%%capture log
## supervised tuning of the stacked network
## 1. stack the two DAE into a MLP
## supervised-training the MLP

mlp_yaml = r"""!obj:pylearn2.train.Train {
    dataset: &train !pkl: '../data/train_mnist.pkl',
    model: !obj:pylearn2.models.mlp.MLP {
        batch_size: 100,
        layers: [
                 !obj:pylearn2.models.mlp.PretrainedLayer {
                     layer_name: 'h1',
                     layer_content: !pkl: "../models/dae_l1.pkl"
                 },
                 !obj:pylearn2.models.mlp.PretrainedLayer {
                     layer_name: 'h2',
                     layer_content: !pkl: "../models/dae_l2.pkl"
                 },
                 !obj:pylearn2.models.mlp.Softmax {
                     max_col_norm: 1.9365,
                     layer_name: 'y',
                     n_classes: 10,
                     irange: .005
                 }
                ],
        nvis: 784
    },
    algorithm: !obj:pylearn2.training_algorithms.sgd.SGD {
        learning_rate: .05,
        learning_rule: !obj:pylearn2.training_algorithms.learning_rule.Momentum {
            init_momentum: .5,
        },
        monitoring_dataset:
            {
                'valid' : !pkl: '../data/valid_mnist.pkl',
            },
        cost: !obj:pylearn2.costs.mlp.Default {},
        termination_criterion: !obj:pylearn2.termination_criteria.And {
            criteria: [
                !obj:pylearn2.termination_criteria.MonitorBased {
                    channel_name: "valid_y_misclass",
                    prop_decrease: 0.,
                    N: 100
                },
                !obj:pylearn2.termination_criteria.EpochCounter {
                    max_epochs: 50
                }
            ]
        },
        update_callbacks: !obj:pylearn2.training_algorithms.sgd.ExponentialDecay {
            decay_factor: 1.00004,
            min_lr: .000001
        }
    },
    extensions: [
        !obj:pylearn2.training_algorithms.learning_rule.MomentumAdjustor {
            start: 1,
            saturate: 250,
            final_momentum: .7
        }
    ],
    save_path: "../models/mlp.pkl",
    save_freq: 1
}
"""

mlp_yaml = yaml_parse.load(mlp_yaml)


%time mlp_yaml.main_loop()

In [14]:
print log.stdout[-500:]

n: 1.93520071282
	valid_y_max_max_class: 0.999997682992
	valid_y_mean_max_class: 0.98003508451
	valid_y_min_max_class: 0.548868565573
	valid_y_misclass: 0.0203
	valid_y_nll: 0.0668058268363
	valid_y_row_norms_max: 0.545912116268
	valid_y_row_norms_mean: 0.264345579985
	valid_y_row_norms_min: 0.101699705657
Saving to ../models/mlp.pkl...
Saving to ../models/mlp.pkl done. Time elapsed: 0.929557 seconds
Saving to ../models/mlp.pkl...
Saving to ../models/mlp.pkl done. Time elapsed: 0.927043 seconds



In [15]:
## make use of trained model
## You need to do all the geeky stuff here to convert it back to Theano
import theano.tensor as T
import theano
softmax_model = cPickle.load(open("../models/mlp.pkl"))
X = softmax_model.get_input_space().make_theano_batch()
y = softmax_model.fprop(X)
ylabel = T.argmax(y, axis = 1)
predict = theano.function([X], ylabel)

yhat = predict(test_mnist.X)
ytarget = cPickle.load(open("../data/mnist.pkl"))[-1][1]
print "Accuracy on test data: ", np.mean(yhat == ytarget)

Accuracy on test data:  0.98


## TODO - simplify pylearn2 interface

## Use Pylearn2 to Implement word/image vector fusion
- [source of both idea and code](https://github.com/mganjoo/zslearning)
- it is a standard 1-hidden layer MLP with customized cost function
- the data we use here is that: X (image vectors from DeCaff), Y (word vectors from word2vec)

In [1]:
from pandas import HDFStore
store = HDFStore("../data/cifa_XY.hd5")
X, y = store["/X"].get_values(), store["/Y"].get_values()
print X.shape, y.shape
from sklearn.cross_validation import train_test_split
train_X, XX, train_y, yy = train_test_split(X, y, test_size = 20000, )
valid_X, test_X, valid_y, test_y = train_test_split(XX, yy, test_size = 10000)
print train_X.shape, train_y.shape, valid_X.shape, valid_y.shape, test_X.shape, test_y.shape

(60000, 4096) (60000, 100)
(40000, 4096) (40000, 100) (10000, 4096) (10000, 100) (10000, 4096) (10000, 100)


In [2]:
train_data = DenseDesignMatrix(X = train_X, y = train_y)
valid_data = DenseDesignMatrix(X = valid_X, y = valid_y)
test_data = DenseDesignMatrix(X = test_X, y = test_y)

cPickle.dump(train_data, open("../data/tmp/train_data.pkl", 'w'))
cPickle.dump(valid_data, open("../data/tmp/valid_data.pkl", 'w'))
cPickle.dump(test_data, open("../data/tmp/test_data.pkl", 'w'))

NameError: name 'DenseDesignMatrix' is not defined

In [101]:
hidden_yaml = r"""!obj:pylearn2.models.mlp.Tanh {
  layer_name: 'hidden_layer',
  dim: 200,
  sparse_init: 15,
}"""

output_yaml = r"""!obj:pylearn2.models.mlp.Linear {
  layer_name: 'output_layer',
  dim: 100,
  irange: 0.05,
}"""

mlp_yaml = r"""!obj:pylearn2.models.mlp.MLP {
  layers: [%s, %s, ],
  nvis: 4096,
}""" % (hidden_yaml, output_yaml)

train_data_yaml = r"""&train !pkl: '../data/tmp/train_data.pkl'
"""

algorithm_yaml = r"""!obj:pylearn2.training_algorithms.bgd.BGD {
  batch_size: 1000,
  line_search_mode: 'exhaustive',
  conjugate: 1,
  updates_per_batch: 10,
  monitoring_dataset: {
    'train' : *train,
    'valid' : !pkl: '../data/tmp/valid_data.pkl',
    },
    termination_criterion: !obj:pylearn2.termination_criteria.And {
      criteria: [
        !obj:pylearn2.termination_criteria.MonitorBased {
          channel_name: 'valid_y_mse',
          },
        !obj:pylearn2.termination_criteria.EpochCounter {
          max_epochs: 10, }, ],
    },
}"""

extension_yaml = r"""[
  !obj:pylearn2.train_extensions.best_params.MonitorBasedSaveBest {
    channel_name: 'valid_y_mse',
    save_path: "../models/mlp_best.pkl"
    },]"""

driver_yaml = r"""!obj:pylearn2.train.Train {
  dataset: %s,
  model: %s,
  algorithm: %s,
  save_path: "../models/mlp.pkl",
  save_freq: 1
}""" % (train_data_yaml, mlp_yaml, algorithm_yaml, )



#yaml_parse.load(hidden_yaml)
#yaml_parse.load(output_yaml)
#yaml_parse.load(mlp_yaml)
#yaml_parse.load(train_data_yaml)
#yaml_parse.load(algorithm_yaml)
#yaml_parse.load(extension_yaml)
%time driver = yaml_parse.load(driver_yaml)
%time driver.main_loop()

CPU times: user 16 s, sys: 2.52 s, total: 18.5 s
Wall time: 18.5 s
compiling begin_record_entry...
compiling begin_record_entry done. Time elapsed: 0.124294 seconds
Monitored channels: 
	ave_grad_mult
	ave_grad_size
	ave_step_size
	total_seconds_last_epoch
	train_hidden_layer_col_norms_max
	train_hidden_layer_col_norms_mean
	train_hidden_layer_col_norms_min
	train_hidden_layer_max_x_max_u
	train_hidden_layer_max_x_mean_u
	train_hidden_layer_max_x_min_u
	train_hidden_layer_mean_x_max_u
	train_hidden_layer_mean_x_mean_u
	train_hidden_layer_mean_x_min_u
	train_hidden_layer_min_x_max_u
	train_hidden_layer_min_x_mean_u
	train_hidden_layer_min_x_min_u
	train_hidden_layer_range_x_max_u
	train_hidden_layer_range_x_mean_u
	train_hidden_layer_range_x_min_u
	train_hidden_layer_row_norms_max
	train_hidden_layer_row_norms_mean
	train_hidden_layer_row_norms_min
	train_objective
	train_output_layer_col_norms_max
	train_output_layer_col_norms_mean
	train_output_layer_col_norms_min
	train_output_layer_

KeyError: 'valid_y_mse'

In [6]:
driver_yaml = r"""!obj:pylearn2.train.Train {
    dataset: &train !pkl: '../data/tmp/train_data.pkl',
    model: !obj:pylearn2.models.mlp.MLP {
        layers: [
                 !obj:pylearn2.models.mlp.RectifiedLinear {
                     layer_name: 'h0',
                     dim: 1200,
                     irange: .05,
                     max_col_norm: 1.9365,
                 },
                 !obj:pylearn2.models.mlp.RectifiedLinear {
                     layer_name: 'h1',
                     dim: 1200,
                     irange: .05,
                     max_col_norm: 1.9365,
                 },
                 !obj:pylearn2.models.mlp.LinearGaussian {
                     init_bias: !obj:pylearn2.models.mlp.mean_of_targets { 
                       dataset: *train },
                     init_beta: !obj:pylearn2.models.mlp.beta_from_targets { 
                       dataset: *train },
                     min_beta: 1.,
                     max_beta: 100.,
                     beta_lr_scale: 1.,
                     dim: 100,
                     # max_col_norm: 1.9365,
                     layer_name: 'y',
                     irange: .005
                 }
                ],
        nvis: 4096,
    },
    algorithm: !obj:pylearn2.training_algorithms.bgd.BGD {
        line_search_mode: 'exhaustive',
        batch_size: 1024,
        conjugate: 1,
        reset_conjugate: 0,
        reset_alpha: 0,
        updates_per_batch: 10,
        monitoring_dataset:
            {
                'train' : *train,
                'valid' : !pkl: '../data/tmp/valid_data.pkl',
            },
        termination_criterion: !obj:pylearn2.termination_criteria.MonitorBased {
            channel_name: "valid_y_mse",
            prop_decrease: 0.,
            N: 100
        },
    },
    extensions: [
        !obj:pylearn2.train_extensions.best_params.MonitorBasedSaveBest {
             channel_name: 'valid_y_mse',
             save_path: '../models/mlp_best.pkl'
        },
    ],
    save_freq: 1,
    save_path: '../models/mlp.pkl',
}"""

%time driver = yaml_parse.load(driver_yaml)
%time driver.main_loop()

CPU times: user 14.3 s, sys: 2.24 s, total: 16.5 s
Wall time: 16.5 s
compiling begin_record_entry...
compiling begin_record_entry done. Time elapsed: 0.220236 seconds
Monitored channels: 
	ave_grad_mult
	ave_grad_size
	ave_step_size
	total_seconds_last_epoch
	train_h0_col_norms_max
	train_h0_col_norms_mean
	train_h0_col_norms_min
	train_h0_max_x_max_u
	train_h0_max_x_mean_u
	train_h0_max_x_min_u
	train_h0_mean_x_max_u
	train_h0_mean_x_mean_u
	train_h0_mean_x_min_u
	train_h0_min_x_max_u
	train_h0_min_x_mean_u
	train_h0_min_x_min_u
	train_h0_range_x_max_u
	train_h0_range_x_mean_u
	train_h0_range_x_min_u
	train_h0_row_norms_max
	train_h0_row_norms_mean
	train_h0_row_norms_min
	train_h1_col_norms_max
	train_h1_col_norms_mean
	train_h1_col_norms_min
	train_h1_max_x_max_u
	train_h1_max_x_mean_u
	train_h1_max_x_min_u
	train_h1_mean_x_max_u
	train_h1_mean_x_mean_u
	train_h1_mean_x_min_u
	train_h1_min_x_max_u
	train_h1_min_x_mean_u
	train_h1_min_x_min_u
	train_h1_range_x_max_u
	train_h1_range_x

KeyboardInterrupt: 

In [7]:
driver_yaml = r"""!obj:pylearn2.train.Train {
    dataset: &train !pkl: '../data/tmp/train_data.pkl',
    model: !obj:pylearn2.models.mlp.MLP {
        layers: [
                 !obj:pylearn2.models.mlp.Tanh {
                     layer_name: 'hidden_layer',
                     dim: 200,
                     irange: 0.05,
                 },
                 !obj:pylearn2.models.mlp.LinearGaussian {
                     init_bias: !obj:pylearn2.models.mlp.mean_of_targets { 
                       dataset: *train },
                     init_beta: !obj:pylearn2.models.mlp.beta_from_targets { 
                       dataset: *train },
                     min_beta: 1.,
                     max_beta: 100.,
                     beta_lr_scale: 1.,
                     dim: 100,
                     # max_col_norm: 1.9365,
                     layer_name: 'y',
                     irange: .005
                 }
                ],
        nvis: 4096,
    },
    algorithm: !obj:pylearn2.training_algorithms.bgd.BGD {
        line_search_mode: 'exhaustive',
        batch_size: 1024,
        conjugate: 1,
        reset_conjugate: 0,
        reset_alpha: 0,
        updates_per_batch: 10,
        monitoring_dataset:
            {
                'train' : *train,
                'valid' : !pkl: '../data/tmp/valid_data.pkl',
            },
        termination_criterion: !obj:pylearn2.termination_criteria.MonitorBased {
            channel_name: "valid_y_mse",
            prop_decrease: 0.,
            N: 100
        },
    },
    extensions: [
        !obj:pylearn2.train_extensions.best_params.MonitorBasedSaveBest {
             channel_name: 'valid_y_mse',
             save_path: '../models/mlp_best.pkl'
        },
    ],
    save_freq: 1,
    save_path: '../models/mlp.pkl',
}"""

%time driver = yaml_parse.load(driver_yaml)
%time driver.main_loop()

CPU times: user 15.1 s, sys: 2.45 s, total: 17.5 s
Wall time: 17.5 s
compiling begin_record_entry...
compiling begin_record_entry done. Time elapsed: 0.164418 seconds
Monitored channels: 
	ave_grad_mult
	ave_grad_size
	ave_step_size
	total_seconds_last_epoch
	train_hidden_layer_col_norms_max
	train_hidden_layer_col_norms_mean
	train_hidden_layer_col_norms_min
	train_hidden_layer_max_x_max_u
	train_hidden_layer_max_x_mean_u
	train_hidden_layer_max_x_min_u
	train_hidden_layer_mean_x_max_u
	train_hidden_layer_mean_x_mean_u
	train_hidden_layer_mean_x_min_u
	train_hidden_layer_min_x_max_u
	train_hidden_layer_min_x_mean_u
	train_hidden_layer_min_x_min_u
	train_hidden_layer_range_x_max_u
	train_hidden_layer_range_x_mean_u
	train_hidden_layer_range_x_min_u
	train_hidden_layer_row_norms_max
	train_hidden_layer_row_norms_mean
	train_hidden_layer_row_norms_min
	train_objective
	train_y_beta_max
	train_y_beta_mean
	train_y_beta_min
	train_y_col_norms_max
	train_y_col_norms_mean
	train_y_col_norms_

KeyboardInterrupt: 